In [1]:
import os
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_model_analysis as tfma

from tfx import components
from tfx.proto import example_gen_pb2, trainer_pb2,pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

In [2]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "movies-rec"
SERVING_MODEL_DIR = "serving_model_dir"

In [3]:
DATA_ROOT = "data/movie"

In [4]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [5]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
    ])
)

example_gen = components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output
)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 13
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
statistics_gen = components.StatisticsGen(
    examples=example_gen.outputs["examples"]
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 14
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [7]:
context.show(statistics_gen.outputs["statistics"])

In [8]:
schema_gen = components.SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 15
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [9]:
example_validator = components.ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 16
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [10]:
context.show(example_validator.outputs["anomalies"])

In [11]:
TRANSFORM_MODULE_FILE = "movie_transform.py"

In [15]:
transform = components.Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipeline\Transform\transform_graph\18\.temp_path\tftransform_tmp\90eab23652aa41a4a51d285496971a14\assets


INFO:tensorflow:Assets written to: pipeline\Transform\transform_graph\18\.temp_path\tftransform_tmp\90eab23652aa41a4a51d285496971a14\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 18
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [17]:
context.show(transform.outputs["post_transform_schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'genres_xf',BYTES,required,,-
'title_xf',BYTES,required,,-


In [18]:
TRAINER_MODULE_FILE = "movie_trainer.py"